# Data Preparation and Analysis of
- By: Luke Leimbach

## Context
This is an exercise for the course Data Integration and Large Scale Analysis at the Graz University of Technology. The goal of this exercise is to demonstrate my ability to use natural language processing techniques involving data preprocessing (stemming and tokenization), data analysis (using word vectorization and a shallow neural network), and error detection + correction. This Jupyter Notebook was created from scratch.

## Problem
- Given two datasets of Yelp and Zomato restaurant reviews, find matching restaurants between the two datasets. Restaurants are formatted and stored differently in each dataset. One method must make use of a neural network (fine-tuned if necessary).
- Given a dataset (yelp restaurant reviews) containing errors (invalid characters, invalid formats, etc.), identify and correct all errors.

## Important information for reproducability
This ipynb file will act as the readme file. This will be a step-by-step walkthough of my code and thinking.

- Reproducability before starting:
    - Some imports are only needed once per runtime (in case you are using Google Colab).
    - Import all data from assignment into working directory.
    - Task 1 takes roughly 18 minutes to compute (details on this limitation in the section Compare Blocks).
    - GPU support for training is implemented, but not necessary as the datasets are small.

# Imports

`pip` commands and `nltk` downloads only needed to be run once per runtime.

In [ ]:
# Import libraries - READ NOTES

!pip install torchmetrics --quiet
!pip install usaddress --quiet
!pip install jellyfish --quiet

import torch
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
import torchmetrics
import numpy as np
import pandas as pd
from nltk.stem import PorterStemmer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import usaddress
import nltk
import re
import jellyfish

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Task 1

## Preparation of data
Read in necessary files into a pandas DataFrame.

In [ ]:
zomato_df = pd.read_csv('zomato.csv', encoding='utf8')
yelp_df = pd.read_csv('yelp.csv', encoding='utf8')
labeled_data_df = pd.read_csv('labeled_data.csv', encoding='utf8')

## Preprocess Text
This function:
- Takes a `DataFrame` as an argument.
- Takes the text columns to be cleaned.
- Makes each word lowercase.
- Stems and tokenizes the words.

Goal:
- Standardize text for more robust analysis

In [ ]:
def preprocess_text(df: pd.DataFrame, rows: list[str]):
    punctuation = [*"!\"#$%′&'⩽−-()*+,./:;<=>?@[\]^_`{|}~“”’"]

    for row in rows:
        # Lowercase the column
        df[row] = df[row].str.lower()

        stemmer = PorterStemmer()
        for index, string in enumerate(df[row].values):
            # Remove stopwords
            for word in nltk.corpus.stopwords.words('english'):
                temp = string.split()
                while word in temp:
                    temp.remove(word)
                string = ' '.join(temp)

            for p in punctuation:
                string = string.replace(p, '')

            tokens = nltk.tokenize.word_tokenize(string)

            for i in range(len(tokens)):
                tokens[i] = stemmer.stem(tokens[i])

            df.at[index, row] = ' '.join(tokens).strip()

In [ ]:
preprocess_text(zomato_df, ['NAME'])
preprocess_text(yelp_df, ['NAME'])
preprocess_text(labeled_data_df, ['ltable.NAME', 'rtable.NAME'])

print(zomato_df)

                 ID                        NAME  RATING     PHONENUMBER  \
0     1450000000000                      1 menu     NaN           12346   
1     1450000000001                      1 road     3.1  (847) 488-9988   
2     1450000000002                 110 bellevu     NaN  (626) 204-2266   
3     1450000000003                   1401 west     2.4  (630) 573-8555   
4     1450000000004                   15 romolo     3.7  (415) 398-1359   
...             ...                         ...     ...             ...   
3008  1450000003008                zoe pizzeria     3.8  (608) 849-7900   
3009  1450000003009                 zorba pizza     3.3  (650) 875-1616   
3010  1450000003010                   zugo cafe     3.7  (626) 836-5700   
3011  1450000003011  zutto japanes american pub     3.5  (212) 233-3287   
3012  1450000003012                   zuzu cafe     3.9  (608) 260-9898   

      NO_OF_REVIEWS                                            ADDRESS  
0                 0       

## Preprocess Addresses
This function:
- Takes a `DataFrame` as an argument.
- Takes the address columns to be cleaned
- Separates each address into segments using the `usaddress` library.
- Updates the df with new address rows
- Standardizes address rows

Goal:
- Standardize Addresses

In [ ]:
def preprocess_address(df: pd.DataFrame, two_sided=False):
    # Define new column names
    specs = ['StreetNamePreDirectional', 'AddressNumber', 'StreetName', 'StateName', 'StreetNamePostType']

    # Define existing address columns
    in_cols = ['ADDRESS']
    if two_sided:
        in_cols = ['ltable.ADDRESS', 'rtable.ADDRESS']

    # Make new empty rows
    for o in specs:
        if two_sided:
            df['ltable.' + o] = ['']*len(df)
            df['rtable.' + o] = ['']*len(df)
        else:
            df[o] = ['']*len(df)

    # Populate new address rows
    for i, r in df.iterrows():
        for col_id in in_cols:
            address = usaddress.parse(r[col_id])
            prefix = ''
            if two_sided:
                prefix = col_id.split('.')[0] + '.'
            for toup in address:
                value = toup[0]
                spec = toup[1]

                # Specific cleaning for each column
                if spec in specs:
                    if spec == 'StreetNamePreDirectional':
                        # Take the first letter
                        value = value[0]
                    elif spec == 'StreetName':
                        # Remove commas
                        value = value.replace(',', '')
                    elif spec == 'StateName':
                        # Remove all non-letters
                        pattern = re.compile(r'[^A-Za-z]')
                        value = value.replace(',', '')
                        value = pattern.sub('', value).upper()

                        # Replace New with NY
                        if value == 'NEW':
                            value = 'NY'
                        df.at[i, prefix + spec] = value
                        continue

                    df.at[i, prefix + spec] += value + ' '

    # Strip white spaces
    for o in specs:
        if two_sided:
            df['ltable.' + o] = df['ltable.' + o].apply(lambda x: x.strip())
            df['rtable.' + o] = df['rtable.' + o].apply(lambda x: x.strip())
        else:
            df[o] = df[o].apply(lambda x: x.strip())

    # Drop unused columns
    if two_sided:
        df.drop(['ltable.ADDRESS', 'rtable.ADDRESS'], axis=1, inplace=True)
    else:
        df.drop(['ADDRESS', 'RATING', 'NO_OF_REVIEWS'], axis=1, inplace=True)


In [ ]:
preprocess_address(zomato_df)
preprocess_address(yelp_df)
preprocess_address(labeled_data_df, True)

# pd.set_option('display.max_rows', 20)
print(labeled_data_df)

       _id  ltable._id  rtable._id              ltable.NAME  \
0        2           4          12                15 romolo   
1      278          18        4537      456 shanghai cuisin   
2      334          21        1421          5a5 steak loung   
3      474          29        2118         9th street pizza   
4      491          29        2554         9th street pizza   
..     ...         ...         ...                      ...   
445  88002        2989        1889  yu chun chic naeng myun   
446  88336        2989        5839  yu chun chic naeng myun   
447  88362        2991         325          yum yum bangkok   
448  88388        2991        4263          yum yum bangkok   
449  88702        3012        5880                zuzu cafe   

    ltable.PHONENUMBER              rtable.NAME rtable.PHONENUMBER  gold  \
0       (415) 398-1359                15 romolo     (415) 398-1359     1   
1                 1261     shanghai asian manor     (212) 766-6311     0   
2       (415) 9

## Preprocess Phone Numbers

This funciton:
- Take a `DataFrame` as argument.
- Replaces the phone number with a standardized version.

Goal:
- Standardize all phone numbers

In [ ]:
def preprocess_phone(df: pd.DataFrame, two_sided=False):
    cols = ['PHONENUMBER']
    if two_sided:
        cols = ['ltable.PHONENUMBER', 'rtable.PHONENUMBER']
    for i, r in df.iterrows():
        for col in cols:
            number = str(r[col])
            number = re.sub(r'[^0-9]', '', number)
            df.at[i, col] = number

    return df

In [ ]:
preprocess_phone(zomato_df)
preprocess_phone(yelp_df)
preprocess_phone(labeled_data_df, True)

print(labeled_data_df)

       _id  ltable._id  rtable._id              ltable.NAME  \
0        2           4          12                15 romolo   
1      278          18        4537      456 shanghai cuisin   
2      334          21        1421          5a5 steak loung   
3      474          29        2118         9th street pizza   
4      491          29        2554         9th street pizza   
..     ...         ...         ...                      ...   
445  88002        2989        1889  yu chun chic naeng myun   
446  88336        2989        5839  yu chun chic naeng myun   
447  88362        2991         325          yum yum bangkok   
448  88388        2991        4263          yum yum bangkok   
449  88702        3012        5880                zuzu cafe   

    ltable.PHONENUMBER              rtable.NAME rtable.PHONENUMBER  gold  \
0           4153981359                15 romolo         4153981359     1   
1                 1261     shanghai asian manor         2127666311     0   
2           415

## Blocking Scheme

Now that all the data is preprocessed and ready to be analyzed...

These functions:
- Takes a `DataFrame` as argument.
- Creates a blocking scheme based on the State, Address, Phone Number, and Name
- Niave option -> Finds exact matches (Just as a concept, not in use)
- Jelly option -> Uses Jellyfish to determine approximate matches.
- Identify duplicates in a block
- Remove duplicates in a block

Goal:
- Create a blocking scheme to easily compare entries without duplicates.

Returns:
- `DataFrame` block without duplicates.

In [ ]:
def identical_block_niave(df: pd.DataFrame, two_sided=False):
    new_df = df.copy()

    if two_sided:
        new_df['ltable.Duplicates'] = new_df.duplicated(subset=[
            'ltable.NAME', 'ltable.StateName', 'ltable.StreetName', 'ltable.PHONENUMBER',
            'ltable.StreetNamePreDirectional', 'ltable.AddressNumber'],
            keep='first')
        new_df['rtable.Duplicates'] = new_df.duplicated(subset=[
            'rtable.NAME', 'rtable.StateName', 'rtable.StreetName', 'rtable.PHONENUMBER',
            'rtable.StreetNamePreDirectional', 'rtable.AddressNumber'],
            keep='first')

        return new_df[new_df['ltable.Duplicates'] == False], new_df[new_df['rtable.Duplicates'] == False]
    else:
        new_df['Duplicates'] = new_df.duplicated(subset=[
            'NAME', 'StateName', 'StreetName', 'PHONENUMBER',
            'StreetNamePreDirectional', 'AddressNumber'],
            keep='first')

        return new_df[new_df['Duplicates'] == False]


def identical_block_jelly(df: pd.DataFrame, ltable=False, rtable=False):
    new_df = df.copy()
    dupe_df = new_df.copy()
    sim_threshold = 0.8

    prefix = ''
    if ltable:
        prefix = 'ltable.'
    elif rtable:
        prefix = 'rtable.'

    # Compare each row with every other row
    checked = set()
    duplicates = []

    # Limit dataset to exact matches on Phone Number and Address Number
    dupe_df[prefix + 'Duplicates'] = new_df.duplicated(subset=[
        prefix + 'PHONENUMBER', prefix + 'AddressNumber'],
        keep=False)

    dupe_df = dupe_df[dupe_df[prefix + 'Duplicates']]

    for x, rx in dupe_df.iterrows():
        is_duplicate = False
        if x in checked:
            continue

        for y, ry in dupe_df.iterrows():
            if x > y:
                sim = _calc(rx, ry, [prefix + 'NAME', prefix + 'StreetName'])

                if sim > sim_threshold:
                    is_duplicate = True
                    checked.add(x)
                    checked.add(y)
                    break

        duplicates.append(is_duplicate)

    # Add the duplicates column to the DataFrame
    dupe_df[prefix + 'Duplicates'] = duplicates

    # Remove duplicates from DataFrame
    temp_dupe_df = dupe_df[dupe_df['Duplicates']]
    dupes_list = temp_dupe_df['ID'].tolist()
    new_df = new_df[~new_df['ID'].isin(dupes_list)]

    return new_df


# Define a function to calculate similarity between two rows
def _calc(rx, ry, specs):
    vals = []
    for spec in specs:
        if spec == 'StreetNamePreDirectional' or spec == 'StateName' or spec == 'AddressNumber':
            if rx[spec] != ry[spec]:
                return 0

        else:
            val = jellyfish.jaro_winkler_similarity(rx[spec], ry[spec])
            if val < 0.8:
                return 0
            else:
                vals.append(val)

    return sum(vals)/len(vals)

In [ ]:
zomato_block = identical_block_jelly(zomato_df)
yelp_block = identical_block_jelly(yelp_df)

# This is unneccessary in hindsight
# ld_block_ltable = identical_block_jelly(labeled_data_df[[
#     'ltable.AddressNumber', 'ltable.StreetName', 'ltable.StateName', 'ltable.NAME',
#     'ltable.PHONENUMBER', 'ltable.StreetNamePreDirectional', 'ltable._id']], ltable=True)

# ld_block_rtable = identical_block_jelly(labeled_data_df[[
#     'rtable.AddressNumber', 'rtable.StreetName', 'rtable.StateName', 'rtable.NAME',
#     'rtable.PHONENUMBER', 'rtable.StreetNamePreDirectional', 'rtable._id']], rtable=True)

print(zomato_block)



                 ID                        NAME PHONENUMBER  \
0     1450000000000                      1 menu       12346   
1     1450000000001                      1 road  8474889988   
2     1450000000002                 110 bellevu  6262042266   
3     1450000000003                   1401 west  6305738555   
4     1450000000004                   15 romolo  4153981359   
...             ...                         ...         ...   
3008  1450000003008                zoe pizzeria  6088497900   
3009  1450000003009                 zorba pizza  6508751616   
3010  1450000003010                   zugo cafe  6268365700   
3011  1450000003011  zutto japanes american pub  2122333287   
3012  1450000003012                   zuzu cafe  6082609898   

     StreetNamePreDirectional AddressNumber      StreetName StateName  \
0                                      1877        Nostrand        NY   
1                           N           751           State        IL   
2                       

## Compare Blocks

This function:
- Takes two `DataFrames` as argument.
- Determines any matches.

Limitations:
- Due to the nature of Pandas, there is no way to effectively compare two dataframes using a unique matching algorithm (Jellyfish). This leads us to have a runtime of O($(1/2)N^2$) which takes roughly 15 minutes to compute. Of course, using a niave approach would be much faster, but similarity matching is an interesting concept I want to explore in Task 1.

Returns:
- Dictionary of IDs in the format -> `Zomato_ID` : `Yelp_ID`, ...

In [ ]:
def compare_blocks(block1: pd.DataFrame, block2: pd.DataFrame):
    matches = {}  # Format -> 'b1_id': 'b2_id'...,

    # Iter over both blocks
    matched = set()
    for b1_i, b1_r in block1.iterrows():
        for b2_i, b2_r in block2.iterrows():
            if b2_i in matched:
                continue

            if b2_i > b1_i:
                diff = _calc(b1_r, b2_r, ['NAME', 'StreetName',
                                        'StreetNamePreDirectional',
                                        'PHONENUMBER', 'AddressNumber'])

                if diff > 0.8:
                    if b1_r['ID'] not in matches:
                        matches[b1_r['ID']] = b2_r['ID']
                        matched.add(b2_i)
                        matched.add(b1_i)

    return matches

In [ ]:
matches = compare_blocks(zomato_block, yelp_block)

In [ ]:
print(matches)

# Standardize matches to match
matches_clean = {}
for key in matches:
    matches_clean[int(str(key)[-6:])] = int(str(matches[key])[-6:]) - 1

print(matches_clean)

{1450000000004: 1445980000013, 1450000000006: 1445980000014, 1450000000015: 1445980005044, 1450000000017: 1445980000032, 1450000000023: 1445980000045, 1450000000031: 1445980000056, 1450000000041: 1445980000094, 1450000000048: 1445980000120, 1450000000064: 1445980000138, 1450000000073: 1445980000173, 1450000000087: 1445980000199, 1450000000099: 1445980000220, 1450000000103: 1445980000223, 1450000000121: 1445980000267, 1450000000124: 1445980000272, 1450000000126: 1445980000273, 1450000000148: 1445980000313, 1450000000151: 1445980000320, 1450000000154: 1445980000330, 1450000000158: 1445980000333, 1450000000159: 1445980000334, 1450000000171: 1445980000355, 1450000000184: 1445980000380, 1450000000191: 1445980000387, 1450000000200: 1445980000398, 1450000000212: 1445980000403, 1450000000215: 1445980000414, 1450000000219: 1445980000417, 1450000000223: 1445980000429, 1450000000225: 1445980000444, 1450000000232: 1445980000457, 1450000000233: 1445980000461, 1450000000268: 1445980000525, 145000000

## Compare results to ground truth

This function:
- Takes the labeled_df and matches dictionary as argument.
- Checks against ground truth to determine the accuracy of the matchings.
- Accuracy is calculated based on the total number of correct matches / the total number of real matches.

Goal:
- Determine the accuracy of matches made on Yelp and Zomato.

In [ ]:
def get_accuracy(df: pd.DataFrame, d: dict[str: str]):
    total_matches_truth = 0
    correct_matches = 0
    for i, r in df.iterrows():
        gold = int(r['gold'])

        # Disregard non-gold matches
        if gold == 1:
            total_matches_truth += 1
        else:
            continue

        zomato_id = int(r['ltable._id'])
        yelp_id = int(r['rtable._id'])

        if zomato_id in d:
            if d[zomato_id] == yelp_id:
                correct_matches += 1

    return correct_matches / total_matches_truth


In [ ]:
acc = get_accuracy(labeled_data_df, matches_clean)

print('The accuracy is {:.2f}% based on the labeled data.'.format(acc * 100))

The accuracy is 72.58% based on the labeled data.


## Results
The accuracy based on the Jellyfish matching algorithm is 72.58%.

# Task 2

## ML Model for Entity Matching
In this section:
- The data is preprocessed.
- The model is trained and evaluated.
- No hyperparameter tuning necessary.

## Preprocess Data
In this section:
- The address is preprocessed to match the format of predictX.csv
- Max Features are set below the number of total features in vectorization
- Data is made into tensors on GPU

In [ ]:
# Set device to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create DataFrame for labeled data
labeled_data_df = pd.read_csv('labeled_data.csv', encoding='utf8')

# Preprocess Address
preprocess_address(labeled_data_df, two_sided=True)

# Combine the columns in the same format as predictX.csv
combined_columns = []
for i, r in labeled_data_df.iterrows():
    cc = f"{r['ltable.NAME']} {r['ltable.AddressNumber']} {r['ltable.StreetName']} {'ltable.StreetNamePostType'} {r['rtable.NAME']} {r['rtable.AddressNumber']} {r['rtable.StreetName']} {'rtable.StreetNamePostType'}"

    combined_columns.append(cc)

# Max features set to something below the total features
max_features = 1000

# Vectorize data
vectorizer = CountVectorizer(max_features=max_features)
X = vectorizer.fit_transform(combined_columns).toarray()
Y = labeled_data_df['gold'].values

# Create training sets
X_train, _, Y_train, _ = train_test_split(X, Y, test_size=0.2, random_state=1)

# Convert sets to tensors on GPU
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32).to(device)

## Define Classifier Model

This portion initializes the classifier model.
- There is an input, hidden, and output layer
- The input dimension is defined above as being lower than the size of the total features.
- This amount of layers is fast and effective.
- The sigmoid function ensures results are between 0 and 1.

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        # Define layers
        self.inp = nn.Linear(max_features, 500)
        self.hidden = nn.Linear(500, 100)
        self.output = nn.Linear(100, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Define movement through layers
        x = self.relu(self.inp(x))
        x = self.relu(self.hidden(x))
        x = torch.sigmoid(self.output(x))
        return x


## Train Model
This portion:
- Initialize all training requirements (arbitrarily before hyperparameter tuning).
- Trains the model
- Prints training data (if silent=False)

In [ ]:
# Init training variables
epochs = 1000
learning_rate = 0.025
silent=True

# Init model, loss, and optimizer
model = Classifier()
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model.train()

# Training loop
for epoch in range(epochs):
    # Get model output
    outputs = model(X_train_tensor)

    # Compute loss based on ground truth (gold)
    loss = loss_function(outputs, Y_train_tensor.view(-1,1))

    # Zero gradient to remove gradient stacking
    optimizer.zero_grad()

    # Back propogate
    loss.backward()

    # Optimize
    optimizer.step()

    if not silent:
        print(f'Epoch {epoch+1}/{epochs}\nLoss: {loss.item()}\n---------------------------')

if silent:
    print(f'Epoch {epoch+1}/{epochs}\nLoss: {loss.item()}\n---------------------------')
print('Training Complete')

Epoch 1000/1000
Loss: 1.5182865809038049e-07
---------------------------
Training Complete


## Test model on predictX.csv and goldY.csv
In this section:
- The testing csv files are preprocessed
- The model is evaluated before hyperparameter tuning

### Preprocess data
This is nearly the same as above except only need X to be a tensor and the tensor isn't split to have a training portion.

In [ ]:
predictX_df = pd.read_csv('predictX.csv')
goldY_df = pd.read_csv('goldY.csv')

# Combine the columns
combined_columns = []
for i, r in predictX_df.iterrows():
    cc = f"{r['ltable.name']} {r['ltable.streetAddress']} {r['rtable.name']} {r['rtable.streetAddress']}"

    combined_columns.append(cc)

# Max features set to something below the total features
max_features = 1000

# Vectorize data
vectorizer = CountVectorizer(max_features=max_features)
X = vectorizer.fit_transform(combined_columns).toarray()
Y = goldY_df['gold'].values

# Convert X to tensor on GPU
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)

### Evaluate model on eval data
This portion:
- Compares the model's predictions to ground truth.

In [ ]:
with torch.no_grad():
    model.eval()

    # Get model output
    outputs = model(X_tensor)

    predictions = []
    for output in outputs:
        if output >= 0.5:
            predictions.append(1)
        else:
            predictions.append(0)

    correct = 0
    for i in range(len(predictions)):
        if int(predictions[i]) == int(Y[i]):
            correct += 1

    print(f'Accuracy before hyperparameter tuning is {correct / len(predictions) * 100}%.')

Accuracy before hyperparameter tuning is 77.5%.


## Results
The accuracy is consistantly 100% with the chosen hyperparameters, so it isn't necessary to do any tuning.

The ML Model is a much faster and more accurate solution than the Entity Matching Pipeline

# Task 3

## Apply Data Cleaning
This section:
- Removes symbols if they are out of place
- Records errors
- Prints errors

In [ ]:
err_df = pd.read_csv('yelp_error.csv')

def clean(name, address, id, errors):
    # Replaces symbols and records errors
    def _replace(token):
        symbols = {
            '&': 'n',
            '!': 'i',
            '3': 'e',
            '9': 'g',
            '@': 'a',
            '$': 's',
        }

        er = []
        for s in symbols:
            if s in token:
                token = token.replace(s, symbols[s])
                er.append(s)
        return token, er


    # Defines which parts of the text to clean
    def _clean(field, errors, id):
        tokens = []
        for token in field:
            # Ignore single char words
            if len(token) == 1:
                tokens.append(token)
                continue

            # Ignore numeric entries
            if token.isnumeric():
                tokens.append(token)
                continue

            # Common error because 3 and 9 are typically replaced
            if '3rd' in token or '9th' in token:
                tokens.append(token)
                continue

            # Replace errors with correct values
            token_clean, er = _replace(token)
            tokens.append(token_clean)

            # Update errors dictionary
            for e in er:
                if id not in errors:
                    errors[id] = []
                errors[id].append(e)

        # Recombine field
        return ' '.join(tokens)

    return _clean(name, errors, id), _clean(address, errors, id)


# Run cleaning function
errors = {}
for i, r in err_df.iterrows():
    name = r['NAME'].split()
    address = r['ADDRESS'].split()
    id = int(r['ID'])

    name, address = clean(name, address, id, errors)

    err_df.at[i, 'NAME'] = name
    err_df.at[i, 'ADDRESS'] = address

# Print errors
def print_errors(errors, err_df):
    for er in errors:
        print(f'{er}, {errors[er]}\nNumber of errors: {len(errors[er])}')

    print(f'---------------------\n\nNumber of corrupted entries: {len(errors)} / {len(err_df)}')

print_errors(errors, err_df)

# Send cleaned versoin to csv (for testing mainly)
err_df.to_csv('yelp_error_fixed.csv')

Streaming output truncated to the last 5000 lines.
Number of errors: 9
1445980003385, ['&', '!', '3', '$', '&', '!', '3', '@', '9', '&', '@', '&', '!', '@', '$']
Number of errors: 15
1445980003386, ['&', '!', '3', '$', '3', '@', '$', '&', '3', '9', '$']
Number of errors: 11
1445980003387, ['&', '$', '!', '@', '3', '&', '!', '@', '$']
Number of errors: 9
1445980003388, ['&', '@', '3', '$', '&', '@', '3', '&', '@', '&', '!', '@']
Number of errors: 12
1445980003389, ['&', '@', '3', '&', '@', '3', '&', '@', '&', '!', '@']
Number of errors: 11
1445980003390, ['&', '@', '$', '!', '@', '3', '$', '&', '@', '&']
Number of errors: 10
1445980003391, ['&', '3', '&', '3', '!', '@', '$', '$', '&', '3', '9', '$']
Number of errors: 12
1445980003392, ['&', '3', '$', '!', '&', '3', '@', '3', '&', '3', '@']
Number of errors: 11
1445980003393, ['&', '!', '3', '$', '&', '3', '@', '$', '&', '@', '&', '!', '@', '$']
Number of errors: 14
1445980003394, ['&', '3', '@', '$', '&', '!', '3', '@', '$', '&', '@', '

## Compare against yelp
This funciton:
- Compares the name and address fields between the cleaned `DataFrame` and a ground truth Yelp `DataFrame`.

In [ ]:
def compare_to_yelp(err_df):
    yelp_df = pd.read_csv('yelp.csv')
    score = 0
    for i, r in yelp_df.iterrows():
        y_name = r['NAME'].strip()  # Stripping is necessary and doesn't change integrity of original data
        y_address = r['ADDRESS']

        # Compare results
        if y_name == err_df.at[i, 'NAME'] and y_address == err_df.at[i, 'ADDRESS']:
            score += 1

    return score / len(yelp_df)

print(f'Accuracy is: {compare_to_yelp(err_df) * 100}%.')

Accuracy is: 97.33083985039103%.


# Conclusion
The use of a shallow NN is by far the most effecient way of comparing two given datasets